In [1]:
import pandas as pd
import openpyxl
import numpy as np
import os
import requests
import re
import spacy
import gc
import zipfile

from collections import Counter
# from imblearn.under_sampling import RandomUnderSampler

excel_dir  = r"D:\Arquivos de Programas HDD\IMDEPA ROLAMENTOS IMPORTACAO E COMA\Canal de Equipe - Suprimentos\IMP\1. SOURCING\8. PROJETOS\PYTHON\NCM_CLASSIFIER\xlsx"
feather_dir  = r"D:\Arquivos de Programas HDD\IMDEPA ROLAMENTOS IMPORTACAO E COMA\Canal de Equipe - Suprimentos\IMP\1. SOURCING\8. PROJETOS\PYTHON\NCM_CLASSIFIER\ftrs"
os.makedirs(excel_dir, exist_ok=True)
os.makedirs(feather_dir, exist_ok=True)

In [2]:
url = "https://www.ime.usp.br/~pf/dicios/br-sem-acentos.txt"
response = requests.get(url)
with open("br-sem-acentos.txt", "wb") as f:
    content = response.content.decode("ASCII")
    lingua_pt_br = content.split()

url = "https://raw.githubusercontent.com/stopwords-iso/stopwords-pt/master/stopwords-pt.txt"
response = requests.get(url)
with open("stopwords-pt.txt", "wb") as f:
    content = response.content.decode("utf-8")
    stopwords_pt_br = content.split()

decoded_stp_wrds = []

for word in stopwords_pt_br:
    word = unidecode(word)
    decoded_stp_wrds.append(word)
    decoded_stp_wrds.append("marca")

stopwords_pt_br = set(decoded_stp_wrds)

del decoded_stp_wrds
del url
del response
del content
lingua_pt_br = [palavra for palavra in lingua_pt_br if palavra not in stopwords_pt_br]
del stopwords_pt_br

lingua_pt_br = set(lingua_pt_br)

dirty_words = set()

def limpa_trilho(coluna):
    global dirty_words
    clean_words = []
    for word in str(coluna).split():
        word = str(word).lower()
        word = unidecode(word)
        word = re.sub(r'[^A-Za-z0-9° ]', '', word)
        if word in lingua_pt_br:
            clean_words.append(word)
        else:
            if word not in dirty_words:
                dirty_words.add(word)
            else:
                pass
        # clean_words.append(word)
    return " ".join(clean_words)
    del clean_words

In [4]:
prefixes = ['COD.NCM', 'NCM', 'DESCRICAO_DETALHADA', 'DESCRICAO DO PRODUTO']

with alive_bar(len(os.listdir(excel_dir)), force_tty=True) as bar:
    with os.scandir(excel_dir) as excels:
        for entry in excels:
            if entry.is_file():
                df_feather = pd.DataFrame()
                print(entry.name)
                with pd.ExcelFile(f'{excel_dir}/{entry.name}') as xls:
                    df_feather = pd.read_excel(xls, header=0, usecols="A:Y", nrows=150000)
                selec_cols = [col for col in df_feather.columns if any(col.startswith(prefix) for prefix in prefixes)]
                df_feather = df_feather[selec_cols]
                NCM_col = [col for col in df_feather.columns if 'ncm' in col.lower()]
                Desc_col = [col for col in df_feather.columns if 'descricao' in col.lower()]
                df_feather = df_feather.rename(columns={NCM_col[0]: 'NCM', Desc_col[0]: 'Descricao Item'})
                df_feather = df_feather.dropna()
                df_feather['tags'] = df_feather['Descricao Item'].apply(limpa_trilho)

                extension = entry.name.rsplit('.', 1)
                feather_name = entry.name.replace('.xlsx', '.fea')
                df_feather.to_feather(f'{feather_dir}/{feather_name}')
                gc.collect()
            bar()
        gc.collect()

'21 11 - POS 8483 - ÍNDIA.xlsx'

,NCM,tags
0,84834010,ex caixas multiplicacao rotacao transmissao ap...


|████████████████████████████████████████| 103/103 [100%] in 8:10.0 (0.21/s)    


In [6]:
feather = len(pd.read_feather(f'{feather_dir}/21 01 - CAP 40 - CHINA.fea'))
excel = len(pd.read_excel(f'{excel_dir}/21 01 - CAP 40 - CHINA.xlsx', header=0))

print(f'Feather: {feather}')
print(f'Excel: {excel}')

Feather: 34558
Excel: 34558


In [7]:
df_chicken = pd.DataFrame()

with alive_bar(len(os.listdir(feather_dir)), force_tty=True) as bar:
    with os.scandir(feather_dir) as feathers:
        for entry in feathers:
            if entry.is_file():
                df_feather = pd.DataFrame()
                ftr = f'{feather_dir}/{entry.name}'
                df_feather = pd.read_feather(ftr)

                # X = df_feather.drop('NCM', axis=1)
                # y = df_feather['NCM']
                # print(Counter(y).items())
                # sampling_strategy = {category: max(int(round(count*0.7,0)), 1) for category, count in Counter(y).items()}
                # rus = RandomUnderSampler(sampling_strategy=sampling_strategy, random_state=42)
                # X_resampled, y_resampled = rus.fit_resample(X, y)
                # df_feather = pd.concat([X_resampled, y_resampled], axis=1)

                df_chicken = pd.concat([df_chicken, df_feather], ignore_index=True)

df_chicken = df_chicken[(df_chicken['NCM'] >= 84820000) & (df_chicken['NCM'] < 84830000)]
display(df_chicken)

on 0: 21 01 - CAP 40 - ALEMANHA.fea                                             
on 0: 21 01 - CAP 40 - CHINA.fea                                                
on 0: 21 01 - CAP 40 - INDIA.fea                                                
on 0: 21 01 - CAP 73 - CHINA.fea                                                
on 0: 21 01 - CAP 87 - CHINA.fea                                                
on 0: 21 01 - POS 8482 - CHINA.fea                                              
on 0: 21 01 - POS 8482 - ÍNDIA.fea                                              
on 0: 21 01 - POS 8483 - CHINA.fea                                              
on 0: 21 02 - CAP 40 - ALEMANHA.fea                                             
on 0: 21 02 - CAP 40 - CHINA.fea                                                
on 0: 21 02 - CAP 40 - INDIA.fea                                                
on 0: 21 02 - CAP 73 - CHINA.fea                                                
on 0: 21 02 - CAP 87 - CHINA

,NCM,tags
0,40011000,latex borracha natural
1,40012990,unidade manopla uso aeronave
2,40012990,membrana pinca utilizacao maquina lavanderia i...
3,40012990,unidades manopla uso aeronave
4,40021110,composto borracha usado limpeza molde industri...
...,...,...
2401812,84836090,articulacao esferica fabricada aco carbono tra...
2401813,84833029,inferior torre
2401814,84834090,plastico eixo transmissao cedulas modulo bilhe...
2401815,84833029,rolamento incorporado diametro interno motores...


In [9]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

vectorizer = CountVectorizer()
doc_term_matrix = vectorizer.fit_transform(df_chicken['tags'])

X_train, X_test, y_train, y_test = train_test_split(doc_term_matrix,
                                                    df_chicken['NCM'],
                                                    random_state=42,
                                                    test_size=0.2)

model = MultinomialNB()
model.fit(X_train, y_train)
pred = model.predict(X_test)

MemoryError: Unable to allocate 189. GiB for an array with shape (2401817, 10568) and data type int64

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

print(f'Accuracy: {accuracy_score(y_test, pred)}')
matrix = confusion_matrix(y_test, pred)
matrix.diagonal()/matrix.sum(axis=1)
class_accuracies = matrix.diagonal() / matrix.sum(axis=1)
class_labels = model.classes_
per_class_accuracy = dict(zip(class_labels, class_accuracies))

acc_df = pd.DataFrame(list(per_class_accuracy.items()), columns=['Class', 'Accuracy'])
acc_df.to_excel('/content/acc.xlsx', index=False)
display(acc_df)

In [ ]:
import joblib

joblib.dump(model, '/content/trained_model.joblib')
joblib.dump(vectorizer, '/content/vectorizer.joblib')

def load_model_and_vectorizer(model_path, vectorizer_path):
    model = joblib.load(model_path)
    vectorizer = joblib.load(vectorizer_path)
    return model, vectorizer

def predict_ncm(product_description, model, vectorizer):
    product_description = limpa_trilho(product_description)
    description_vectorized = vectorizer.transform([product_description])
    predicted_ncm = model.predict(description_vectorized)
    return predicted_ncm[0]

In [ ]:
loaded_model, loaded_vectorizer = load_model_and_vectorizer('trained_model.joblib', 'vectorizer.joblib')

product_description = input("Inform the detailed product description: ")

predicted_ncm = predict_ncm(product_description, loaded_model, loaded_vectorizer)
print("Suggested NCM code:", predicted_ncm)